# Membuat layanan inferensi real time

Setelah melatih model prediktif, Anda dapat menyebarkannya sebagai layanan real time yang dapat digunakan klien untuk mendapatkan prediksi dari data baru.

## Menghubungkan ke ruang kerja Anda

Untuk memulai, hubungkan ke ruang kerja Anda.

> **Catatan**: Jika Anda belum membuat sesi yang terautentikasi dengan langganan Azure, Anda akan diminta untuk mengautentikasi dengan mengklik tautan, memasukkan kode autentikasi, dan masuk ke Azure.

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## Melatih dan mendaftarkan model

Sekarang mari kita melatih dan mendaftarkan model.

In [ ]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name="mslearn-train-diabetes")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('data/diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

## Menyebarkan model sebagai layanan web

Anda telah melatih dan mendaftarkan model pembelajaran mesin yang mengklasifikasikan pasien berdasarkan kemungkinan mereka menderita diabetes. Model ini dapat digunakan dalam lingkungan produksi seperti operasi dokter di mana hanya pasien yang dianggap berisiko yang perlu menjalani uji klinis diabetes. Untuk mendukung skenario ini, Anda akan menyebarkan model sebagai layanan web.

Pertama, mari kita tentukan model apa yang telah Anda daftarkan di ruang kerja.

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

Baiklah, sekarang mari kita dapatkan model yang ingin kita sebarkan. Secara default, jika kita menentukan nama model, versi terbaru akan dikembalikan.

In [ ]:
model = ws.models['diabetes_model']
print(model.name, 'version', model.version)

Kami akan membuat layanan web untuk menghosting model ini, dan ini akan memerlukan beberapa kode dan file konfigurasi; jadi mari kita buat folder untuk itu.

In [ ]:
import os

# Create a folder for the deployment files
deployment_folder = './diabetes_service'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

# Set path for scoring script
script_file = 'score_diabetes.py'
script_path = os.path.join(deployment_folder,script_file)

Layanan web tempat kami menyebarkan model akan memerlukan beberapa kode Python untuk memuat data input, mendapatkan model dari ruang kerja, dan menghasilkan serta mengembalikan prediksi. Kami akan menyimpan kode ini dalam *skrip entri* (sering disebut *skrip penilaian*) yang akan menyebarkan ke layanan web.

Skrip terdiri dari dua fungsi:

- **init**: Fungsi ini dipanggil saat layanan diinisialisasi, dan umumnya digunakan untuk memuat model. Perhatikan bahwa skrip penilaian menggunakan variabel lingkungan **AZUREML_MODEL_DIR** untuk menentukan folder tempat model disimpan.
- **run**: Fungsi ini dipanggil setiap kali aplikasi klien mengirimkan data baru, dan umumnya digunakan untuk menyimpulkan prediksi dari model.

In [ ]:
%%writefile $script_path
import json
import joblib
import numpy as np
import os

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'diabetes_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Layanan web akan dihosting dalam sebuah kontainer, dan kontainer tersebut perlu menginstal semua dependensi Python yang diperlukan saat diinisialisasi. Dalam hal ini, kode penilaian kami memerlukan **scikit-learn** dan beberapa paket khusus Azure Machine Learning yang digunakan oleh layanan web penilaian, jadi kami akan membuat lingkungan yang menyertakan ini. Kemudian kita akan menambahkan lingkungan tersebut ke *konfigurasi inferensi* bersama dengan skrip penilaian, dan menentukan *konfigurasi penyebaran* untuk kontainer tempat lingkungan dan skrip akan dihosting.

Kami kemudian dapat menyebarkan model sebagai layanan berdasarkan konfigurasi ini.

> **Informasi Lebih Lanjut**: Untuk detail selengkapnya tentang penyebaran model, dan opsi untuk lingkungan eksekusi target, lihat [dokumentasi](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where).

Penyebaran akan memakan waktu karena pertama kali mengeksekusi proses untuk membuat citra kontainer, lalu menjalankan proses untuk membuat layanan web berdasarkan citra. Saat penyebaran berhasil diselesaikan, Anda akan melihat status **Sehat**.

In [ ]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Configure the scoring environment
service_env = Environment(name='service-env')
python_packages = ['scikit-learn', 'azureml-defaults', 'azure-ml-api-sdk']
for package in python_packages:
    service_env.python.conda_dependencies.add_pip_package(package)
inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=service_env)

# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model as a service
print('Deploying model...')
service_name = "diabetes-service"
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)
print(service.state)

Semoga penyebaran berhasil dan Anda dapat melihat status **Sehat**. Jika tidak, Anda dapat menggunakan kode berikut untuk mendapatkan log layanan untuk membantu Anda memecahkan masalah.

In [ ]:
print(service.get_logs())

# If you need to make a change and redeploy, you may need to delete unhealthy service using the following code:
#service.delete()

Lihat ruang kerja Anda di [Azure Machine Learning Studio](https://ml.azure.com) dan lihat halaman **Titik akhir**, yang menampilkan layanan yang disebarkan di ruang kerja Anda.

Anda juga dapat mengambil nama layanan web di ruang kerja Anda dengan menjalankan kode berikut:

In [ ]:
for webservice_name in ws.webservices:
    print(webservice_name)

## Menggunakan layanan web

Dengan layanan yang disebarkan, sekarang Anda dapat menggunakannya dari aplikasi klien.

In [ ]:
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

Anda juga dapat mengirim beberapa pengamatan pasien ke layanan, dan mendapatkan kembali prediksi untuk setiap pengamatan.

In [ ]:
import json

# This time our input is an array of two feature arrays
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes.
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Kode di atas menggunakan Azure Machine Learning SDK untuk terhubung ke layanan web berbasis kontainer dan menggunakannya untuk menghasilkan prediksi dari model klasifikasi diabetes Anda. Dalam produksi, model kemungkinan akan dikonsumsi oleh aplikasi bisnis yang tidak menggunakan Azure Machine Learning SDK, tetapi hanya membuat permintaan HTTP ke layanan web.

Mari kita tentukan URL tempat aplikasi ini harus mengirimkan permintaannya:

In [ ]:
endpoint = service.scoring_uri
print(endpoint)

Sekarang setelah Anda mengetahui URI titik akhir, aplikasi dapat dengan mudah membuat permintaan HTTP, mengirim data pasien dalam format JSON, dan menerima kembali kelas yang diprediksi.

In [ ]:
import requests
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Anda telah menyebarkan layanan web Anda sebagai layanan Azure Container Instance (ACI) yang tidak memerlukan autentikasi. Hal ini bagus untuk pengembangan dan pengujian, tetapi untuk produksi, Anda harus mempertimbangkan untuk menyebarkan ke kluster Azure Kubernetes Service (AKS) dan mengaktifkan autentikasi berbasis token. Ini akan membutuhkan permintaan REST untuk menyertakan header **Otorisasi**.

## Menghapus layanan

Saat Anda tidak lagi membutuhkan layanan Anda, Anda harus menghapusnya untuk menghindari timbulnya biaya yang tidak perlu.

In [ ]:
service.delete()
print ('Service deleted.')

Untuk informasi selengkapnya tentang menerbitkan model sebagai layanan, lihat [dokumentasi](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where)